In [ ]:
from scipy.sparse import diags, block_diag, eye
from scipy.sparse.linalg import spsolve, splu

import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "seaborn"

# Thermal explosion 

## Model with fuel consumption neglected

$$
\partial_\tau \theta - {\displaystyle 1 \over \displaystyle \lambda} \partial_{zz} \theta = \exp(\theta)
$$

### The fonction $\Psi$

In [ ]:
x = np.linspace(0,20,1000)
Psi = np.exp(-x/2) * np.arccosh(np.exp(x/2))
# Psi = 1/2 * np.exp(-x/2) * np.log( (1+np.sqrt(1-np.exp(-x))) / (1-np.sqrt(1-np.exp(-x))) )
Psi_max = np.max(Psi)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=Psi))
fig.update_layout(xaxis_title="x", yaxis_title="Psi", title=f"Maximal value of Psi: {Psi_max:.4f}")
fig.show()

### Integration with IMEX scheme

In [ ]:
def imex(tini, tend, nt, yini, a, f, nt_eval=None, param=None):
    
    if param is not None:
          f = lambda x, f=f : f(x, *param)

    dt = (tend-tini) / (nt-1)
    
    lu = splu(eye(yini.size, format="csc") - dt*a)

    if nt_eval is None: nt_eval = (nt,)
        
    y = np.zeros((len(nt_eval), yini.size))
        
    yn = np.copy(yini)

    it_eval = 0 
    for it in range(1,nt):
        yn = lu.solve(yn + dt*f(yn))
        if (it+1 in nt_eval):
            y[it_eval] = yn 
            it_eval = it_eval+1
            
    if nt_eval is None: y[-1] = yn

    return y

In [ ]:
## model parameters
xmin = 0
xmax = 2
lamb = 0.87

## diffusion part 
nx = 99
dx = (xmax-xmin) / (nx+1)
cst = 1./(lamb*dx*dx)

# laplacian matrix with homegenous Dirichlet conditions
diag0 = np.repeat(-2, nx)
diag1 = np.repeat(1, nx-1)
lap = diags([diag1, diag0, diag1], [-1, 0, 1], format="csc")
lap = cst*lap

## reaction part
def f(y):
    return np.exp(y)

## time integration
tini = 0
tend = 10
nt = 1001
dt = (tend-tini) / (nt-1)

# integration
theta_ini = np.zeros(nx)
theta = imex(tini, tend, nt, theta_ini, lap, f)
theta = imex(tini, tend, nt, theta_ini, lap, f, nt_eval=(nt//4, nt//2, (3*nt)//4, nt))

# output
x = np.linspace(xmin, xmax, nx+2)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=np.concatenate(([0],theta_ini,[0])), name='it = 0'))
fig.add_trace(go.Scatter(x=x, y=np.concatenate(([0],theta[0],[0])),  name='it = nt/4'))
fig.add_trace(go.Scatter(x=x, y=np.concatenate(([0],theta[1],[0])),  name='it = nt/2'))
fig.add_trace(go.Scatter(x=x, y=np.concatenate(([0],theta[2],[0])),  name='it = 3nt/4'))
fig.add_trace(go.Scatter(x=x, y=np.concatenate(([0],theta[-1],[0])), name='it = nt'))

fig.update_layout(xaxis_title="x", yaxis_title="theta")
fig.show()

## Model with fuel consumption

$$
\left\{\begin{aligned}
\partial_\tau \theta - {\displaystyle 1 \over \displaystyle \lambda} \partial_{zz} \theta & = \exp(\theta)\, Y\\
\partial_\tau Y - {\displaystyle 1 \over \displaystyle \lambda} \partial_{zz} Y & = - \varepsilon \exp(\theta)\, Y
\end{aligned}\right.
$$

### Integration with IMEX scheme (order 2 Neumann conditions for Y)

In [ ]:
## model parameters
xmin = 0
xmax = 2
lamb = 0.87
eps = 0.002

## diffusion part 
nxib = 101 # nb space points including boundaries 
nx = nxib-2
dx = (xmax-xmin) / (nx+1)
cst = 1./(lamb*dx*dx)

# laplacian matrix with homegenous Dirichlet conditions
diag0 = np.repeat(-2, nx)
diag1 = np.repeat(1, nx-1)
lapd = diags([diag1, diag0, diag1], [-1, 0, 1], format="csc")
lapd = cst*lapd

# laplacian matrix with homegenous Neumman conditions (order 2)
diag3 = np.repeat(-2, nxib)
diag4 = np.repeat([2., 1., 2], [1, nxib-2, 1])
lapn = diags([diag4[1:],  diag3, diag4[:-1]], [-1, 0, 1])
lapn = cst*lapn

laplap = block_diag((lapd, lapn))
##print(laplap.todense())

## reaction part
def f2(sol, eps):
    ntot = sol.size
    nx = (ntot-2)//2
    theta = sol[:nx]
    Y = sol[nx:]
    tmp = np.exp(theta)*Y[1:-1]
    return np.concatenate((tmp, -eps*np.concatenate(([Y[0]], tmp, [Y[-1]]))))

## time integration
tini = 0
tend = 10
nt = 1001
dt = (tend-tini) / (nt-1)

# integration
x = np.linspace(xmin, xmax, nxib)
sol_ini = np.zeros(nx+nxib)
sol_ini[nx:] = np.ones(nxib)
#sol_ini[nx:] = np.ones(nxib) + 0.5*(x-1)
sol = imex(tini, tend, nt, sol_ini, laplap, f2, param=(eps,), nt_eval=(nt//4, nt//2, (3*nt)//4, nt))
theta = sol[:,:nx]
Y = sol[:,nx:]

# output
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=x, y=np.concatenate(([0],sol_ini[:nx],[0])), name='it = 0', line_color='rgb(76,114,176)'), col=1, row=1)
fig.add_trace(go.Scatter(x=x, y=sol_ini[nx:], showlegend=False, line_color='rgb(76,114,176)'), col=1, row=2)
fig.add_trace(go.Scatter(x=x, y=np.concatenate(([0],theta[0],[0])), name='it = nt/4', line_color='rgb(221,132,82)'), col=1, row=1)
fig.add_trace(go.Scatter(x=x, y=Y[0], showlegend=False, line_color='rgb(221,132,82)' ), col=1, row=2)
fig.add_trace(go.Scatter(x=x, y=np.concatenate(([0],theta[1],[0])), name='it = nt/2', line_color='rgb(85,168,104)'), col=1, row=1)
fig.add_trace(go.Scatter(x=x, y=Y[1], showlegend=False, line_color='rgb(85,168,104)'), col=1, row=2)
fig.add_trace(go.Scatter(x=x, y=np.concatenate(([0],theta[2],[0])), name='it = 3nt/4', line_color='rgb(196,78,82)'), col=1, row=1)
fig.add_trace(go.Scatter(x=x, y=Y[2], showlegend=False, line_color='rgb(196,78,82)'), col=1, row=2)
fig.add_trace(go.Scatter(x=x, y=np.concatenate(([0],theta[-1],[0])), name='it = nt', line_color='rgb(129,114,179)'), col=1, row=1)
fig.add_trace(go.Scatter(x=x, y=Y[-1], showlegend=False, line_color='rgb(129,114,179)'), col=1, row=2)
fig.update_xaxes(title_text="x", col=1)
fig.update_yaxes(title_text="theta", row=1)
fig.update_yaxes(title_text="Y", row=2)
fig.update_layout(height=1000)
fig.show()

## Model with fuel consumption and without linearization

$$
\left\{\begin{aligned}
\partial_\tau \theta - {\displaystyle 1 \over \displaystyle \lambda} \partial_{zz} \theta & = 
\exp\Big(\frac{\theta}{1+\theta/\beta}\Big)\, Y\\
\partial_\tau Y - {\displaystyle 1 \over \displaystyle \lambda} \partial_{zz} Y & = - \varepsilon \exp\Big(\frac{\theta}{1+\theta/\beta}\Big)\, Y
\end{aligned}\right.
$$

### Integration with IMEX scheme (order 2 Neumann conditions for Y)

In [ ]:
## model parameters
xmin = 0
xmax = 2
lamb = 0.87
eps = 0.002
beta = 30

## diffusion part 
nxib = 101 # nb space points including boundaries 
nx = nxib-2
dx = (xmax-xmin) / (nx+1)
cst = 1./(lamb*dx*dx)

# laplacian matrix with homegenous Dirichlet conditions
diag0 = np.repeat(-2, nx)
diag1 = np.repeat(1, nx-1)
lapd = diags([diag1, diag0, diag1], [-1, 0, 1], format="csc")
lapd = cst*lapd

# laplacian matrix with homegenous Neumman conditions (order 2)
diag3 = np.repeat(-2, nxib)
diag4 = np.repeat([2., 1., 2], [1, nxib-2, 1])
lapn = diags([diag4[1:],  diag3, diag4[:-1]], [-1, 0, 1])
lapn = cst*lapn

laplap = block_diag((lapd, lapn))
##print(laplap.todense())

## reaction part
def f3(sol, eps, beta):
    ntot = sol.size
    nx = (ntot-2)//2
    theta = sol[:nx]
    Y = sol[nx:]
    tmp = np.exp( theta / (1+theta/beta) )*Y[1:-1]
    return np.concatenate((tmp, -eps*np.concatenate(([Y[0]], tmp, [Y[-1]]))))

## time integration
tini = 0
tend = 10
nt = 10001
dt = (tend-tini) / (nt-1)

# integration
x = np.linspace(xmin, xmax, nxib)
sol_ini = np.zeros(nx+nxib)
sol_ini[nx:] = np.ones(nxib)
##sol_ini[nx:] = np.ones(nxib) + 0.5*(x-1)
sol = imex(tini, tend, nt, sol_ini, laplap, f3, param=(eps,beta), nt_eval=(nt//4, nt//2, (3*nt)//4, nt))
theta = sol[:,:nx]
Y = sol[:,nx:]

# output
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=x, y=np.concatenate(([0],sol_ini[:nx],[0])), name='it = 0', line_color='rgb(76,114,176)'), col=1, row=1)
fig.add_trace(go.Scatter(x=x, y=sol_ini[nx:], showlegend=False, line_color='rgb(76,114,176)'), col=1, row=2)
fig.add_trace(go.Scatter(x=x, y=np.concatenate(([0],theta[0],[0])), name='it = nt/4', line_color='rgb(221,132,82)'), col=1, row=1)
fig.add_trace(go.Scatter(x=x, y=Y[0], showlegend=False, line_color='rgb(221,132,82)' ), col=1, row=2)
fig.add_trace(go.Scatter(x=x, y=np.concatenate(([0],theta[1],[0])), name='it = nt/2', line_color='rgb(85,168,104)'), col=1, row=1)
fig.add_trace(go.Scatter(x=x, y=Y[1], showlegend=False, line_color='rgb(85,168,104)'), col=1, row=2)
fig.add_trace(go.Scatter(x=x, y=np.concatenate(([0],theta[2],[0])), name='it = 3nt/4', line_color='rgb(196,78,82)'), col=1, row=1)
fig.add_trace(go.Scatter(x=x, y=Y[2], showlegend=False, line_color='rgb(196,78,82)'), col=1, row=2)
fig.add_trace(go.Scatter(x=x, y=np.concatenate(([0],theta[-1],[0])), name='it = nt', line_color='rgb(129,114,179)'), col=1, row=1)
fig.add_trace(go.Scatter(x=x, y=Y[-1], showlegend=False, line_color='rgb(129,114,179)'), col=1, row=2)
fig.update_xaxes(title_text="x", col=1)
fig.update_yaxes(title_text="theta", row=1)
fig.update_yaxes(title_text="Y", row=2)
fig.update_layout(height=1000)
fig.show()